[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PicselliaTeam/picsellia-training-engine/blob/master/PicselliaTrainingQuickstart.ipynb)

In [ ]:
from picsellia_training.clientv2 import Client
from picsellia_tf2 import pxl_utils
from picsellia_tf2 import pxl_tf
import os
api_token = '60ef4a0b3ab8086709e2a7246cc0d134beb09d8b'
project_token = '265e8fd5-0f26-4ca2-b4c7-5f248a59f560'

experiment = Client.Experiment(api_token=api_token, project_token=project_token, host='http://127.0.0.1:8000/sdk/v2/')

exp = experiment.checkout('model 1')
print(exp)

experiment.dl_annotations()
experiment.dl_pictures()
experiment.generate_labelmap()
experiment.train_test_split()
parameters = {
    "annotation_type": 'rectangle',
    "num_steps": 1000,
    "batch_size": 4,
    "learning_rate": 1e-3
}
experiment.log('parameters', parameters)
pxl_utils.create_record_files(
        dict_annotations=experiment.dict_annotations, 
        train_list=experiment.train_list, 
        train_list_id=experiment.train_list_id, 
        eval_list=experiment.eval_list, 
        eval_list_id=experiment.eval_list_id,
        label_path=experiment.label_path, 
        record_dir=experiment.record_dir, 
        tfExample_generator=pxl_tf.tf_vars_generator, 
        annotation_type=parameters['annotation_type']
        )
    
pxl_utils.edit_config(
        model_selected=experiment.checkpoint_dir, 
        input_config_dir=experiment.config_dir,
        output_config_dir=experiment.config_dir,
        record_dir=experiment.record_dir, 
        label_map_path=experiment.label_path, 
        num_steps=parameters['num_steps'],
        batch_size=parameters['batch_size'],
        learning_rate=parameters['learning_rate'],
        annotation_type=parameters['annotation_type'],
        eval_number = 5,
        )

pxl_utils.train(
        ckpt_dir=experiment.checkpoint_dir, 
        config_dir=experiment.config_dir
    )

pxl_utils.evaluate(
    experiment.metrics_dir, 
    experiment.config_dir, 
    experiment.checkpoint_dir
    )        
pxl_utils.export_graph(
    ckpt_dir=experiment.checkpoint_dir, 
    exported_model_dir=experiment.exported_model_dir, 
    config_dir=experiment.config_dir
    )
pxl_utils.infer(
    experiment.record_dir, 
    exported_model_dir=experiment.exported_model_dir, 
    label_map_path=os.path.join(experiment.base_dir,'label_map.pbtxt'), 
    results_dir=experiment.results_dir, 
    from_tfrecords=True, 
    disp=False
    )

metrics = pxl_utils.tf_events_to_dict('my_new_model2/metrics', 'eval')
logs = pxl_utils.tf_events_to_dict('my_new_model2/checkpoint', 'train')
print(logs)
experiment.store('model-latest', 'my_new_model2/exported_model.zip')
experiment.store('config', 'my_new_model2/config/pipeline.config')
experiment.store('checkpoint-data-latest', 'my_new_model2/checkpoint/ckpt-10.data-00000-of-00001')
experiment.store('checkpoint-index-latest', 'my_new_model2/checkpoint/ckpt-11.index')
experiment.log('logs', logs)
experiment.log('metrics', metrics)


**Welcome To Picsellia Training Quickstart Notebook** 

In this Notebook, you will see how to launch a training from a created experiment on the Platform and log all the evaluation metrics to analyse your trained model.

**Step 1, let's import our python SDK and our tensorflow2 wrapper** 

If you do not have our packages you can run: 
- pip install picsellia picsellia_tf2 


In [ ]:
from picsellia import Client
from picsellia_tf2 import pxl_utils
from picsellia_tf2 import pxl_tf
import os

**Step 2, fetch your experiment parameters from the Picsellia servers**

Every experiment you make has an unique identifier allowing you to retrieve all the necessary informations with one command

In [ ]:
api_token = 'YOU CAN FIND YOUR TOKEN IN YOUR PROFILE PAGE :D '
experiment_id= "YOUR EXPERIMENT ID IS ON THE TOP LEFT CORNER OF YOUR EXPERIMENT"

experiment = Client.Experiment(api_token=api_token, host=host)
exp = experiment.checkout(experiment_id, tree=True, with_file=True)

**Step 3, Data fetching** 

Download all the necessary data for your training:
- Dataset
- Annotations

Then generate the labelmap for your model, based on your annotations 
Finally performing train-test-split to perform training, 
**(default repartition is 0.8 / 0.2 from train/test)**

Once the train test set created, we send the repartition to Picsellia platform in order to visualize it later 

In [ ]:
experiment.dl_annotations()
experiment.dl_pictures()
experiment.generate_labelmap()
experiment.train_test_split()

train_split = {
    'x': experiment.categories,
    'y': experiment.train_repartition,
    'image_list': experiment.train_list_id
}
experiment.log('train-split', train_split, 'bar', replace=True)

test_split = {
    'x': experiment.categories,
    'y': experiment.test_repartition,
    'image_list': experiment.eval_list_id
}
experiment.log('test-split', test_split, 'bar', replace=True)

**Step 4, Pre-processing** 

Now we will create the necessary record files to perform training, and initialize the training with the parameters that you chose for your experiment on Picsellia

In [ ]:
parameters = experiment.get_data(name='parameters')

pxl_utils.create_record_files(
        dict_annotations=experiment.dict_annotations, 
        train_list=experiment.train_list, 
        train_list_id=experiment.train_list_id, 
        eval_list=experiment.eval_list, 
        eval_list_id=experiment.eval_list_id,
        label_path=experiment.label_path, 
        record_dir=experiment.record_dir, 
        tfExample_generator=pxl_tf.tf_vars_generator, 
        annotation_type=parameters['annotation_type']
        )
    
pxl_utils.edit_config(
        model_selected=experiment.checkpoint_dir, 
        input_config_dir=experiment.config_dir,
        output_config_dir=experiment.config_dir,
        record_dir=experiment.record_dir, 
        label_map_path=experiment.label_path, 
        num_steps=parameters["steps"],
        batch_size=parameters['batch_size'],
        learning_rate=parameters['learning_rate'],
        annotation_type=parameters['annotation_type'],
        eval_number = 5,
        )

**Step 5, Training** 

Then just launch training, and go grab a cup of coffee :D 

In [ ]:
pxl_utils.train(
        ckpt_dir=experiment.checkpoint_dir, 
        config_dir=experiment.config_dir
    )


**Step 6, Evaluation**

Now let's run evaluation on your trained model in order to analyse the performances later

In [ ]:
pxl_utils.evaluate(
    experiment.metrics_dir, 
    experiment.config_dir, 
    experiment.checkpoint_dir
    )   

**Step 7, Exporting and Inference**

This part will export your trained model as saved_model in order to use in production or for inference in Picsellia. 

Inference will be performed on several images of your test set and sent to Picsellia platform to visualize some results and share it with your colloborators or community. 

Then all the evaluation metrics will be uploaded to your experiments pages in order to visualize all your graphs :) 

In [ ]:
pxl_utils.export_graph(
    ckpt_dir=experiment.checkpoint_dir, 
    exported_model_dir=experiment.exported_model_dir, 
    config_dir=experiment.config_dir
    )
pxl_utils.infer(
    experiment.record_dir, 
    exported_model_dir=experiment.exported_model_dir, 
    label_map_path=os.path.join(experiment.base_dir,'label_map.pbtxt'), 
    results_dir=experiment.results_dir, 
    from_tfrecords=True, 
    disp=False
    )

metrics = pxl_utils.tf_events_to_dict('{}/metrics'.format(exp.experiment_name), 'eval')
logs = pxl_utils.tf_events_to_dict('{}/checkpoint'.format(exp.experiment_name), 'train')
experiment.store('model-latest')
experiment.store('config')
experiment.store('checkpoint-data-latest')
experiment.store('checkpoint-index-latest')
for variable in logs.keys():
    data = {
        'steps': logs[variable]["steps"],
        'values': logs[variable]["values"]
    }
    experiment.log('-'.join(variable.split('/')), data, 'line', replace=True)
experiment.log('metrics', metrics, 'table', replace=True)

![](https://www.google.com/search?q=google+image&client=ubuntu&hs=Q7r&channel=fs&source=lnms&tbm=isch&sa=X&ved=2ahUKEwii17Hsr9juAhWlnVwKHRUoD0sQ_AUoAXoECBQQAw&biw=2560&bih=931#imgrc=TMmIvimt9rgaYM)